# Import Libraries

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import datetime
import csv
import openpyxl
from openpyxl import Workbook
import pandas as pd

# Create Functions

In [118]:
class Draeger:
    # prepare the input date
    def __init__(self, year, month, day):
        self.output_date=int(str(year)+str(month)+str(day))

    # scrape info about: name of the product and url
    def scrape(self):
        #open chrome in incognito mode
        options = webdriver.ChromeOptions()
        options.add_argument(' -- incognito')
        browser = webdriver.Chrome(chrome_options=options)

        # deal with the first "medical staff?" question
        browser.get('https://www.draeger.com/ja_jp/Hospital/Portfolio')

        # wait for browser to open for 10 sec
        timeout = 10
        try:
            WebDriverWait(browser, timeout).until(
            EC.visibility_of_element_located(
            (By.XPATH, '//*[@id="acceptdisclaimer_ja_jp_hospital"]')
            )
            )
        except TimeoutException:
            print('Timed Out Waiting for page to load')
            browser.quit()

        # Click the yes button
        login_btn=browser.find_element_by_xpath('//*[@id="acceptdisclaimer_ja_jp_hospital"]')
        login_btn.click()
        browser.implicitly_wait(3)

        # Get info
        # Get the URL list
        page_list=browser.find_elements_by_css_selector("div[class='carditem ng-scope']")
        page_url_list= [page.find_element_by_css_selector('figure>a').get_attribute('href') for page in page_list]

        # create a list
        result=[]
        # Go through the url list
        for page_url in page_url_list:
            # go to the URL
            browser.get(page_url)
            # from here, use try and except to get info from different format
            # 1st, get info from table structure
            try:
                tabsection_0=browser.find_element_by_id('tabsection_0') # product section
                category_list=tabsection_0.find_elements_by_css_selector("li[class='navitem ng-scope']") # list of url
                category_url_list=[category.find_element_by_css_selector("a").get_attribute('href') for category in category_list]
                # append the current url to the list
                if len(category_url_list)<0:
                    category_url_list.append(page_url)
                # go through the url list
                for category_url in category_url_list:
                    browser.get(category_url)
                    category_1=browser.find_element_by_id("tabsectionpanel_0_1")
                    print(category_url,category_1.text)
                # # this try is for swiping structure
                # try:
                #     product=browser.find_element_by_css_selector("div[class='tabsectionpanel']").text
                #     print(product,category_count)
                # except Exception:
                #     print('no')
                    #     product_name=product.find_element_by_css_selector("h3[class='ng-binding ng-scope']").text.strip()
                    #     product_url=product.find_elements_by_css_selector("a[class='primarybtn small ng-binding ng-scope ng-isolate-scope']").get_attribute('href')
                    #     print(product_name,product_url)
                    #     result.append([product_name, product_url])
                    # # this exception is for those without swiping structure
                    # except NoSuchElementException:
                    #     product_name=category.find_element_by_css_selector("h3[class='ng-binding ng-scope']").text.strip()
                    #     product_url=product.find_elements_by_css_selector("a[class='primarybtn small ng-binding ng-isolate-scope']").get_attribute('href')
                    #     result.append([product_name, product_url])
    
            # 2nd, get info from non-table structure
            except Exception:
                pass


            # # get the category list
            # category_list=browser.find_elements_by_css_selector("li[class='navitem ng-scope']")
            # category_url_list=[category.find_element_by_css_selector("a").get_attribute('href') for category in category_list]
            # # if there's no url, add the default url
            # if len(page_url_list)<0:
            #     category_url_list.append(page_url)

            # # go through the list
            # for category_url in category_url_list:
            #     # go to the URL
            #     browser.get(category_url)

            #     # Get link and title
            #     product_list=browser.find_elements_by_css_selector('div.swiper-wrapper')
            #     for product in product_list:
            #         product_url=browser.find_elements_by_css_selector("a[class='primarybtn small ng-binding ng-scope ng-isolate-scope']").get_attribute('href')
            #         product_name=browser.find_element_by_css_selector("h3[class='ng-binding ng-scope']").text.strip()
            #         # Append the info to the list
                    # result.append([product_name, product_url])

        # close the browser        
        browser.quit()
        return print(result)

    # store the list into CSV if there is no csv. If not, load the csv and check if there's any product thats not in the older csv.
    def get_new_product(self):
        try:
        # load an old list of product and convert it to a dataframe
            column_names=['product_name','product_url']
            df_product_list=pd.read_csv("Izumi_product_list.csv",names=column_names)

            # get result
            result = self.scrape()
            

            # Find if there's any new products
            new_product_list=[]
            for product in result:
                if not(df_product_list['product_name'].isin([product[0]]).any()):
                    new_product_list.append(product)
            
            # check if there's any new product. If so, append them to the existing product list
            if len(new_product_list)>0:
                with open('Izumi_product_list.csv', 'a') as csvfile:
                    writer = csv.writer(csvfile)
                    for i in range(len(new_product_list)):
                        writer.writerow([new_product_list[i][0],  
                                        new_product_list[i][1]])

            # return a list of new products
            return new_product_list
        
        # if there's no such file, create a new file 
        except FileNotFoundError:
            with open('Izumi_product_list.csv','w') as csvfile:
                result = self.scrape()
                result_len=len(result)
                writer = csv.writer(csvfile)
                for i in range(result_len):
                    writer.writerow([result[i][0],  
                                    result[i][1]])
            # return an empty list
            return []

   
   
    # store new products' info into csv
    def to_csv(self):
        # get new products
        new_product_list=self.get_new_product()
        new_product_list_len = len(new_product_list)

        # check if the result is empty
        if new_product_list_len == 0:
            return

        # get row number
        # try to open the csv file
        try:
            with open('product_info.csv') as csvfile:
                reader = csv.reader(csvfile)
                for row in reader:
                    # the date is already there, dont add anything
                    if (row[0]==self.output_date) and (row[4]=='008'):
                        return print('Already added to csv')

        # if there's no such file, create a new file 
        except FileNotFoundError:
            with open('product_info.csv','w') as csvfile:
                pass
        
        # add new data
        with open('product_info.csv', 'a') as csvfile:
            writer = csv.writer(csvfile)
            for i in range(new_product_list_len):
                writer.writerow([self.output_date, 
                                '人工心肺', 
                                '手術室', 
                                '医療機器管理',
                                5,
                                '泉工医科工業', 
                                '新製品',
                                new_product_list[i][0], 
                                new_product_list[i][1], 
                                1])

# Run the Function

In [119]:
if __name__=='__main__':
    year=2020
    month=9
    day=13
    draeger=Draeger(year,month,day)
    draeger.scrape()
    # draeger.to_csv()

https://www.draeger.com/ja_jp/Hospital/Portfolio/Anaesthesia-Devices#Anesthetic-machines 
https://www.draeger.com/ja_jp/Hospital/Portfolio/Anaesthesia-Devices#Anaesthetic-gas-monitoring 
https://www.draeger.com/ja_jp/Hospital/Portfolio/Anaesthesia-Devices#Decision-support 
https://www.draeger.com/ja_jp/Hospital/Portfolio/Anaesthesia-Devices#Vaporisers 
https://www.draeger.com/ja_jp/Hospital/Portfolio/Ventilation-Respiratory-Monitoring#ICU-ventilation-and-respiratory-monitoring 
https://www.draeger.com/ja_jp/Hospital/Portfolio/Ventilation-Respiratory-Monitoring#Turbine-driven-ventilator 
https://www.draeger.com/ja_jp/Hospital/Portfolio/Ventilation-Respiratory-Monitoring#Neonatal-ventilators 
https://www.draeger.com/ja_jp/Hospital/Portfolio/Medical-Lights#Operating-lights 
https://www.draeger.com/ja_jp/Hospital/Portfolio/Medical-Lights#Examination-lights 
https://www.draeger.com/ja_jp/Hospital/Portfolio/Supply-Systems#Ceiling-supply-units 
https://www.draeger.com/ja_jp/Hospital/Portfolio

In [10]:
options = webdriver.ChromeOptions()
options.add_argument(' -- incognito')
browser = webdriver.Chrome(chrome_options=options)
url_list=[
    'https://www.draeger.com/ja_jp/Hospital/Portfolio/Anaesthesia-Devices#Anesthetic-machines',
    # 'https://www.draeger.com/ja_jp/Hospital/Portfolio/Anaesthesia-Devices#Anaesthetic-gas-monitoring'
]
result=[]
for url in url_list:
    # deal with the first "medical staff?" question
    browser.get(url)

    # wait for browser to open for 10 sec
    timeout = 10
    try:
        WebDriverWait(browser, timeout).until(
        EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="acceptdisclaimer_ja_jp_hospital"]')
        )
        )
    except TimeoutException:
        print('Timed Out Waiting for page to load')
        browser.quit()

    # Click the yes button
    login_btn=browser.find_element_by_xpath('//*[@id="acceptdisclaimer_ja_jp_hospital"]')
    login_btn.click()
    browser.implicitly_wait(3)

    '''from here, scrape info'''
    bullet_list=browser.find_element_by_css_selector("div[class='portfolio-pagination swiper-pagination-clickable swiper-pagination-bullets']").find_elements_by_css_selector('span')
    product_category=browser.find_element_by_id('tabsectionpanel_0_0')
    html_content=browser.find_element_by_id('tabsectionpanel_0_0').get_attribute('innerHTML')
    product_name_list=[]
    product_url_list=[]
    for bullet in bullet_list:
        bullet.click()
        # product_name_temp_list=product_category.find_elements_by_css_selector("h3[class='ng-binding ng-scope'")
        product_url_temp_list=product_category.find_elements_by_css_selector("a[class='primarybtn small ng-binding ng-scope ng-isolate-scope']")
        # for product_name_raw in product_name_temp_list:
        #     product_name=product_name_raw.text.strip()
        #     print(product_name)
        #     if product_name not in product_name_list and product_name!='':
        #         product_name_list.append(product_name)
        for product_url_raw in product_url_temp_list:
            product_url=product_url_raw.get_attribute('href')
            if product_url not in product_url_list:
                product_url_list.append(product_url)
    
    
    browser.get("data:text/html;charset=utf-8,{html_content}".format(html_content=html_content))
    product_name_list=browser.find_elements_by_css_selector('h3')
    for product_name_raw in product_name_list:
        print(type(product_name_raw))
        product_name_list.append(product_name_raw)

    # print(product_name_list)
    # print(product_url_list)




<class 'selenium.webdriver.remote.webelement.WebElement'>
<class 'selenium.webdriver.remote.webelement.WebElement'>
<class 'selenium.webdriver.remote.webelement.WebElement'>
<class 'selenium.webdriver.remote.webelement.WebElement'>
<class 'selenium.webdriver.remote.webelement.WebElement'>
<class 'selenium.webdriver.remote.webelement.WebElement'>
<class 'selenium.webdriver.remote.webelement.WebElement'>
<class 'selenium.webdriver.remote.webelement.WebElement'>
<class 'str'>


AttributeError: 'str' object has no attribute 'text'

In [57]:
str='Fabius plus (ファビウス プラス)'
print(str in product_name_list)

False
